# Setup

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers==4.22.2

!pip install statsmodels

!pip install datasets

!pip install -U tensorflow==2.10 

!nvidia-smi

In [ ]:
# main libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from collections import defaultdict
from tqdm.autonotebook import tqdm
import spacy
import re
import statsmodels
import statsmodels.api as sm
import scipy

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, mean_absolute_percentage_error, r2_score, jaccard_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# specific machine learning functionality
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K
import datasets
from datasets import Dataset
from datasets import load_from_disk

# Transformers
import transformers
from transformers import (
    BertTokenizer, 
    TFBertForSequenceClassification, 
    TFBertForMaskedLM, 
    TFBertModel,
    #create_optimizer,
    #DataCollatorForLanguageModeling,
    #PreTrainedTokenizerFast
)

In [ ]:
# Enable/Disable Eager Execution
# Reference: https://www.tensorflow.org/guide/eager
# TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, 
# without building graphs

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

print("tensorflow version", tf.__version__)
print("keras version", tf.keras.__version__)
print("Eager Execution Enabled:", tf.executing_eagerly())

# Get the number of replicas 
strategy = tf.distribute.MirroredStrategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

devices = tf.config.experimental.get_visible_devices()
print("Devices:", devices)
print(tf.config.experimental.list_logical_devices('GPU'))

print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("All Physical Devices", tf.config.list_physical_devices())

# Better performance with the tf.data API
# Reference: https://www.tensorflow.org/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
word_dir = "/content/drive/MyDrive/"

# Data

In [ ]:
qadataset = datasets.load_dataset("squad")

# EDA

In [ ]:
qadataset

In [ ]:
lengths = [temp['context'].count(' ') for temp in qadataset['train']]
answer_starts = [temp['context'][:temp['answers']['answer_start'][0]].count(' ') for temp in qadataset['train']]
question_lengths = [temp['question'].count(' ') for temp in qadataset['train']]
answer_lengths = [temp['answers']['text'][0].count(' ') for temp in qadataset['train']]

In [ ]:
plt.hist(lengths)
print(np.mean(lengths))
print(np.mean(np.asarray(lengths) > 256))

In [ ]:
plt.hist(answer_starts)
print(np.mean(answer_starts))
print(np.mean(np.asarray(answer_starts) < 256))

In [ ]:
plt.hist(question_lengths)
print(np.mean(question_lengths))
print(np.mean(np.asarray(question_lengths) < 256))

In [ ]:
plt.hist(answer_lengths)
print(np.mean(answer_lengths))
print(np.mean(np.asarray(answer_lengths) < 512 - 384))

In [ ]:
lengths = np.asarray(lengths)
answer_starts = np.asarray(answer_starts)
question_lengths = np.asarray(question_lengths)
answer_lengths = np.asarray(answer_lengths)

# Labeling the Data

In [ ]:
qadataset_train = qadataset['train']
qadataset_test = qadataset['validation']

In [ ]:
def make_negative(example):
    context = example['context']
    answer_start = example['answers']['answer_start'][0]
    sentence_number = context[:answer_start].count(".")
    sentences = context.split(".")
    example['context'] = '.'.join(sentences[:sentence_number] + sentences[sentence_number+1:])
    example['label'] = False
    return example

In [ ]:
def make_positive(example):
    example['label'] = True
    return example

In [ ]:
qadataset_train_label_split = qadataset_train.train_test_split(test_size=0.5, shuffle=True, seed=109)

qadataset_train_positive = qadataset_train_label_split['train']
qadataset_train_negative = qadataset_train_label_split['test']

qadataset_train_negative = qadataset_train_negative.map(make_negative)
qadataset_train_positive = qadataset_train_positive.map(make_positive)

In [ ]:
qadataset_test_label_split = qadataset_test.train_test_split(test_size=0.5, shuffle=True, seed=109)

qadataset_test_positive = qadataset_test_label_split['train']
qadataset_test_negative = qadataset_test_label_split['test']

qadataset_test_positive = qadataset_test_positive.map(make_positive)
qadataset_test_negative = qadataset_test_negative.map(make_negative)

In [ ]:
qadataset_train = datasets.concatenate_datasets([qadataset_train_positive, qadataset_train_negative])
qadataset_test = datasets.concatenate_datasets([qadataset_test_positive, qadataset_test_negative])

In [ ]:
def combine_q_a(example):
    example['text'] = '[CLS] ' + example['question'] + ' [SEP] ' + example['context']
    return example

In [ ]:
qadataset_train = qadataset_train.map(combine_q_a)
qadataset_test = qadataset_test.map(combine_q_a)

# Train QA Classifier

## Tokenization

In [ ]:
### Tokenization parameters
classifier_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(classifier_name, do_lower_case=True)
batch_size = 8 
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
### Tokenization function
def tokenize_for_bert_classifier(df, should_shuffle=False):
    # Tokenization
    X_tokenized = bert_tokenizer.batch_encode_plus(
            df["text"],
            return_tensors='tf',
            add_special_tokens = True,
            return_token_type_ids=True,
            padding='max_length',
            max_length=256,
            return_attention_mask = True,
            truncation='longest_first'
    )
    # Creating TF datasets
    dataset = tf.data.Dataset.from_tensor_slices(((X_tokenized["input_ids"],
                                                   X_tokenized["token_type_ids"],
                                                   X_tokenized["attention_mask"]), 
                                                  df["label"]))
    if should_shuffle:
        buffer_train = len(df["text"])
        dataset = dataset.shuffle(buffer_size=buffer_train)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
qadataset_train = qadataset_train.train_test_split(test_size=0.2, shuffle=True, seed=109)

qadataset_train['validation'] = qadataset_train.pop('test')

classification_training_data = tokenize_for_bert_classifier(qadataset_train['train'], should_shuffle=True)
classification_validation_data = tokenize_for_bert_classifier(qadataset_train['validation'])

In [ ]:
classification_test_data = tokenize_for_bert_classifier(qadataset_test)

## Training

In [ ]:
### BERT Setup
learning_rate = 2e-5
epochs = 5
def get_bert_classifier():
    return TFBertForSequenceClassification.from_pretrained(classifier_name, 
                                                           num_labels=1, from_pt = True)

def get_compiled_bert_classifier(model = None):
    # Free up memory
    K.clear_session()

    # Build the model
    if model is None:
      model = get_bert_classifier()

    # Print the model architecture
    print(model.summary())

    # Optimizer
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
    # Loss
    loss = keras.losses.BinaryCrossentropy(from_logits=True)

    # Compile
    model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=['accuracy'])
    
    return model

In [ ]:
classifier_model = get_compiled_bert_classifier()

In [ ]:
### Train model
epochs = 2
train_model = False
if train_model:
    start_time = time.time()
    training_results = classifier_model.fit(
            classification_training_data,
            validation_data=classification_validation_data,
            epochs=epochs,
            verbose=1)
    execution_time = (time.time() - start_time)/60.0
    print("Training execution time (mins)",execution_time)
    classifier_model.save_pretrained(word_dir + 'Senior Thesis models/model_classifier_bert_6/temp')
else:
    classifier_model = TFBertForSequenceClassification.from_pretrained(word_dir + 'Senior Thesis models/model_classifier_bert_6/temp')

## Evaluation

In [ ]:
### Evaluation Function
def evaluate_bert_classifier(bert_model, dataset, Y_true, only_accuracy = False):
    Y_pred = bert_model.predict(dataset)
    Y_pred = Y_pred['logits'] > 0
    acc = accuracy_score(Y_true, Y_pred)
    print(f"Accuracy: {acc}")
    if only_accuracy:
        return
    f1 = f1_score(Y_true, Y_pred)
    print(f"F1 score: {f1}")
    recall = recall_score(Y_true, Y_pred)
    print(f"Recall score: {recall}")
    precision = precision_score(Y_true, Y_pred)
    print(f"Precision score: {precision}")
    
    Y_pred = np.asarray([x[0] for x in Y_pred])
    
    TN = np.sum((Y_true == Y_pred) & (Y_pred == 0))
    TP = np.sum((Y_true == Y_pred) & (Y_pred == 1))
    
    FN = np.sum((Y_true != Y_pred) & (Y_pred == 0))
    FP = np.sum((Y_true != Y_pred) & (Y_pred == 1))
    
    print(f"TN: {TN}, TP:{TP}, FN:{FN}, FP:{FP}")

In [ ]:
# must have should_shuffle=False for this
evaluate_bert_classifier(classifier_model, classification_training_data, qadataset_train['train']["label"])

In [ ]:
evaluate_bert_classifier(classifier_model, classification_validation_data, qadataset_train['validation']["label"])

In [ ]:
evaluate_bert_classifier(classifier_model, classification_test_data, qadataset_test["label"])